In [14]:
from IPython.core.display import display, HTML, Markdown
import copy

n = 20
# segue una lista di precedenze della forma [u,v], cl significato che u deve essere schedulato oprima di v.
prec_original_instance = [
       [0, 1],   # l'attività 0 deve essere schedulata prima dell'attività 1
       [0, 4],
       [0, 7],
       [0, 15],
       [0, 18],
       [1, 1],
       [1, 4],
       [1, 7],
       [1, 15],
       [1, 18],
]

def visualizza(ordinamento):
    # mathplotlib o networkx o ?
    # visualizziammo il grafo 
    # il problem-solver deve rendersi conto di quali archi sono rivolti all'indietro.
    pass



## Esercizio \[60 pts\]
(CICLI)

In [15]:
# Ecco il grafo in input:
visualizza()

0 0 1 0 1 1 0 1 
0 0 1 0 0 1 1 0 
1 1 0 1 1 0 0 0 
1 1 0 1 0 1 1 0 
0 0 1 0 0 1 0 1 
1 0 1 0 1 1 0 1 


Attualmente ci sono <b>24 luci accese</b>.

__Richieste__:
[15pt] Portati in una configurazione col minor numero possibile di luci accese. Puoi effettuare una mossa alla volta utilizzando le funzioni $flippa\_riga(indice\_riga)$ e $flippa\_colonna(indice\_colonna)$ per spostarti un pò alla volta fino ad una configurazione che reputi ottima.

In [19]:
# Certificato di si, HA CICLI NEGATIVI
# fornisci sottosequenza di archi


Agendo sugli interruttori settati come segue:<br><b>   Interruttori di riga:</b> [0, 0, 1, 1, 0, 0]<br><b>   Interruttori di colonna:</b> [0, 0, 1, 0, 0, 1, 0, 1]

Ti porti dalla configurazione iniziale riportata in $p$ nella seguente configurazione finale:<br>

0 0 0 0 1 0 0 0 
0 0 0 0 0 0 1 1 
0 0 0 0 0 0 1 0 
0 0 0 0 1 1 0 0 
0 0 0 0 0 0 0 0 
1 0 0 0 1 0 0 0 


Attualmente ci sono <b>8 luci accese</b>.

In [ ]:
# Certificato di no, fornire peso che fa si che tutti gli archi abbiano un peso positivo
# i nodi vanno da 1 a n-1, mettimi il valore da sommare per far si di farmi vedere che tutti i cicli sono positivi

lista_pesi=[1 , 5 ,6 , 9, 41]